# **OBJETIVO**
___

 - EXTRAIR DADOS PÚBLICOS DE COMPRAS GOVERNAMENTAIS;
 - TRATAR OS DADOS;
 - ARMAZENAR OS DADOS EM UM ARQUIVO

## EXTRACT
---

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=93468271d4504b53984e5a7ec2404765b5de8f29eaa6cdc756bd7345d8d72b73
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from google.colab import drive
drive.mount('/content/drive')
import requests
import io
import pandas as pd
import pyspark.sql.functions as F

Mounted at /content/drive


In [ ]:
url = "https://dados.educacao.sp.gov.br/sites/default/files/Profici%C3%AAncia%20do%20SARESP%20por%20escola%20de%202023.csv"
#https://dados.educacao.sp.gov.br/sites/default/files/IDESP_ESCOLA_2023_0.csv

In [ ]:
data = requests.get(url, verify=False).content
data

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dados.educacao.sp.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


b'\xef\xbb\xbfDEPADM;DEPBOL;NOMEDEPBOL;CODRMET;REGIAOMETROPOLITANA;SERIE_ANO;COD_PER;PERIODO;CO_COMP;DS_COMP;MEDPROF\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;2\xc2\xba Ano EF;1;MANH\xc3\x83;1;L\xc3\x8dNGUA PORTUGUESA;175,8\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;2\xc2\xba Ano EF;1;MANH\xc3\x83;2;MATEM\xc3\x81TICA;167,7\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;2\xc2\xba Ano EF;2;TARDE;1;L\xc3\x8dNGUA PORTUGUESA;171,1\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;2\xc2\xba Ano EF;2;TARDE;2;MATEM\xc3\x81TICA;163,3\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;2\xc2\xba Ano EF;9;GERAL;1;L\xc3\x8dNGUA PORTUGUESA;172,7\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;2\xc2\xba Ano EF;9;GERAL;2;MATEM\xc3\x81TICA;164,8\r\n1;1;Rede Estadual;1;Regi\xc3\xa3o Metropolitana de S\xc3\xa3o Paulo;5\xc2\xba Ano EF;1;MANH\xc3\x83;1;L\xc3\x8dNG

In [ ]:
parse_data = io.StringIO(data.decode("utf-8"))
print(parse_data)

In [ ]:
df = pd.read_csv(parse_data, sep=';')
df.head()


,DEPADM,DEPBOL,NOMEDEPBOL,CODRMET,REGIAOMETROPOLITANA,SERIE_ANO,COD_PER,PERIODO,CO_COMP,DS_COMP,MEDPROF
0,1,1,Rede Estadual,1,Região Metropolitana de São Paulo,2º Ano EF,1,MANHÃ,1,LÍNGUA PORTUGUESA,"175,8"
1,1,1,Rede Estadual,1,Região Metropolitana de São Paulo,2º Ano EF,1,MANHÃ,2,MATEMÁTICA,"167,7"
2,1,1,Rede Estadual,1,Região Metropolitana de São Paulo,2º Ano EF,2,TARDE,1,LÍNGUA PORTUGUESA,"171,1"
3,1,1,Rede Estadual,1,Região Metropolitana de São Paulo,2º Ano EF,2,TARDE,2,MATEMÁTICA,"163,3"
4,1,1,Rede Estadual,1,Região Metropolitana de São Paulo,2º Ano EF,9,GERAL,1,LÍNGUA PORTUGUESA,"172,7"


In [ ]:
spark = SparkSession.builder.appName("Dados").getOrCreate()

In [ ]:
df = spark.createDataFrame(df)
df.show()

+------+------+-------------+-------+--------------------+---------+-------+-------+-------+-----------------+-------+
|DEPADM|DEPBOL|   NOMEDEPBOL|CODRMET| REGIAOMETROPOLITANA|SERIE_ANO|COD_PER|PERIODO|CO_COMP|          DS_COMP|MEDPROF|
+------+------+-------------+-------+--------------------+---------+-------+-------+-------+-----------------+-------+
|     1|     1|Rede Estadual|      1|Região Metropolit...|2º Ano EF|      1|  MANHÃ|      1|LÍNGUA PORTUGUESA|  175,8|
|     1|     1|Rede Estadual|      1|Região Metropolit...|2º Ano EF|      1|  MANHÃ|      2|       MATEMÁTICA|  167,7|
|     1|     1|Rede Estadual|      1|Região Metropolit...|2º Ano EF|      2|  TARDE|      1|LÍNGUA PORTUGUESA|  171,1|
|     1|     1|Rede Estadual|      1|Região Metropolit...|2º Ano EF|      2|  TARDE|      2|       MATEMÁTICA|  163,3|
|     1|     1|Rede Estadual|      1|Região Metropolit...|2º Ano EF|      9|  GERAL|      1|LÍNGUA PORTUGUESA|  172,7|
|     1|     1|Rede Estadual|      1|Região Metr

## TRANSFORM

In [ ]:
df.printSchema()

root
 |-- DEPADM: long (nullable = true)
 |-- DEPBOL: long (nullable = true)
 |-- NOMEDEPBOL: string (nullable = true)
 |-- CODRMET: long (nullable = true)
 |-- REGIAOMETROPOLITANA: string (nullable = true)
 |-- SERIE_ANO: string (nullable = true)
 |-- COD_PER: long (nullable = true)
 |-- PERIODO: string (nullable = true)
 |-- CO_COMP: long (nullable = true)
 |-- DS_COMP: string (nullable = true)
 |-- MEDPROF: string (nullable = true)



In [ ]:
df.count()

290

In [ ]:
df.columns

['DEPADM',
 'DEPBOL',
 'NOMEDEPBOL',
 'CODRMET',
 'REGIAOMETROPOLITANA',
 'SERIE_ANO',
 'COD_PER',
 'PERIODO',
 'CO_COMP',
 'DS_COMP',
 'MEDPROF']

In [ ]:
df = (df.withColumnRenamed('DEPADM', "DEPTO_ADM")
.withColumnRenamed('DEPBOL', "DEPTO_BOLETIM")
)

In [ ]:
df.columns

['DEPTO_ADM',
 'DEPTO_BOLETIM',
 'NOMEDEPBOL',
 'CODRMET',
 'REGIAOMETROPOLITANA',
 'SERIE_ANO',
 'COD_PER',
 'PERIODO',
 'CO_COMP',
 'DS_COMP',
 'MEDPROF']

In [ ]:
df = df.select([x.lower() for x in df.columns])
df.printSchema()
df.show()

root
 |-- depto_adm: long (nullable = true)
 |-- depto_boletim: long (nullable = true)
 |-- nomedepbol: string (nullable = true)
 |-- codrmet: long (nullable = true)
 |-- regiaometropolitana: string (nullable = true)
 |-- serie_ano: string (nullable = true)
 |-- cod_per: long (nullable = true)
 |-- periodo: string (nullable = true)
 |-- co_comp: long (nullable = true)
 |-- ds_comp: string (nullable = true)
 |-- medprof: string (nullable = true)

+---------+-------------+-------------+-------+--------------------+---------+-------+-------+-------+-----------------+-------+
|depto_adm|depto_boletim|   nomedepbol|codrmet| regiaometropolitana|serie_ano|cod_per|periodo|co_comp|          ds_comp|medprof|
+---------+-------------+-------------+-------+--------------------+---------+-------+-------+-------+-----------------+-------+
|        1|            1|Rede Estadual|      1|Região Metropolit...|2º Ano EF|      1|  MANHÃ|      1|LÍNGUA PORTUGUESA|  175,8|
|        1|            1|Rede Esta

In [ ]:
dir = '/content/drive/MyDrive/xpe/Dados'

In [ ]:
df.write.format("overwrite").csv(f"{dir}/dados.csv")

In [ ]:
spark.read.csv('/content/drive/MyDrive/xpe/Dados/dados.csv').show()

+---+---+-------------+---+--------------------+---------+---+-----+---+-----------------+-----+
|_c0|_c1|          _c2|_c3|                 _c4|      _c5|_c6|  _c7|_c8|              _c9| _c10|
+---+---+-------------+---+--------------------+---------+---+-----+---+-----------------+-----+
|  1|  1|Rede Estadual|  1|Região Metropolit...|2º Ano EF|  1|MANHÃ|  1|LÍNGUA PORTUGUESA|175,8|
|  1|  1|Rede Estadual|  1|Região Metropolit...|2º Ano EF|  1|MANHÃ|  2|       MATEMÁTICA|167,7|
|  1|  1|Rede Estadual|  1|Região Metropolit...|2º Ano EF|  2|TARDE|  1|LÍNGUA PORTUGUESA|171,1|
|  1|  1|Rede Estadual|  1|Região Metropolit...|2º Ano EF|  2|TARDE|  2|       MATEMÁTICA|163,3|
|  1|  1|Rede Estadual|  1|Região Metropolit...|2º Ano EF|  9|GERAL|  1|LÍNGUA PORTUGUESA|172,7|
|  1|  1|Rede Estadual|  1|Região Metropolit...|2º Ano EF|  9|GERAL|  2|       MATEMÁTICA|164,8|
|  1|  1|Rede Estadual|  1|Região Metropolit...|5º Ano EF|  1|MANHÃ|  1|LÍNGUA PORTUGUESA|200,2|
|  1|  1|Rede Estadual|  1|Reg